# Querry data based on ro-Crates

1. This specifically looks at the intermediate steps of the metaGOflow pipeline.
2. `fastap` outputs
3. ...

**Steps:** (for each metaGOflow step)
1. Acess ro-crate metadata file and extract needed data sources
2. Get the data
3. Visualize

In [1]:
import requests
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from utils import init_setup
init_setup()

Platform: local Linux


## Imports

In [2]:
# This needs to be repeated here for the Pannel dashboard to work, WEIRD
# TODO: report as possible bug
import sys
import os
import io

# import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
import panel as pn

from json2txttree import json2txttree

# All low level functions are imported from the momics package
from momics.loader import get_ro_crate_metadata_gh


# Note: This is breaking the panel preview functionality
%load_ext autoreload
%autoreload 2

In [3]:
# TODO: is this nomenclature the final one?
sample_id = "EMOBON_PiEGetxo_Wa_4"
met_json = get_ro_crate_metadata_gh(sample_id)

ro-crate-metadata.json request status 200


In [26]:
## pretty nice tool
print(json2txttree(met_json))

└─  (object)
   ├─ "@context" (object)
   │  ├─ "@vocab" (string)
   │  ├─ "dcat" (string)
   │  ├─ "dct" (string)
   │  ├─ "pcdm" (string)
   │  └─ "skos" (string)
   └─ "@graph" (array)
      └─  (object)
         ├─ "@id" (string)
         ├─ "@type" (string)
         ├─ "conformsTo" (array)
         │  └─  (object)
         │     └─ "@id" (string)
         └─ "about" (object)
            └─ "@id" (string)



## Methods

- I do not really want to code parsing of this html
  - TODO: check for some written analyser of fastap

In [5]:

def extract_data_by_name(metadata, name='FASTP analysis of raw sequence data'):
    for d in metadata['@graph']:
        if 'name' in d.keys() and d['name'] == name:
            data = d
            break
    return data


def extract_all_datafiles(metadata):
    datafiles = []
    for d in metadata['@graph']:
        if 'name' in d.keys() and d['@type'] == 'File':
            data_unit = {}
            data_unit['name'] = d['name']
            try:
                # in MB
                data_unit['sizeMB'] = int(int(d['contentSize'])/1e6)
            except KeyError:
                data_unit['sizeMB'] = 'unknown'

            try:
                data_unit['downloadUrl'] = d['downloadUrl']
            except KeyError:
                data_unit['downloadUrl'] = 'unknown'
            datafiles.append(data_unit)
            
    return datafiles

In [6]:
from IPython.display import display, HTML

In [9]:
# extract all related data files
datafiles = extract_all_datafiles(met_json)
assert len(datafiles) == len([k for k in datafiles if k['sizeMB'] != 'unknown'])
assert len(datafiles) == len([k for k in datafiles if k['downloadUrl'] != 'unknown'])

len(datafiles), len([k for k in datafiles if k['sizeMB'] < 100])

(57, 50)

* this means I have 50 outputs which I can do something about with size < 50 MB

In [23]:
# print all data files which contain word fastq
[k for k in datafiles if 'fast' in k['name'].lower()]

[{'name': 'FASTP analysis of raw sequence data',
  'sizeMB': 0,
  'downloadUrl': 'https://s3.mesocentre.uca.fr/mgf-data-products/files/md5/99/d78d57cf8c0ebc38e15162b0afe37f'},
 {'name': 'FASTA formatted contig sequences',
  'sizeMB': 92,
  'downloadUrl': 'https://s3.mesocentre.uca.fr/mgf-data-products/files/md5/b4/1de79ad2ea5e565064d74362b76563'}]

In [31]:
[k['name' ] for k in datafiles]

['MetaGOflow YAML configuration file',
 'FASTP analysis of raw sequence data',
 'FASTA formatted contig sequences',
 'MetaGOflow configuration in YAML',
 'Trimmed reverse reads QC summary',
 'Merged reads',
 'QC summary of merged reads',
 'Trimmed forward reads',
 'Protein coding nucleotide sequences',
 'MOTUs',
 'Overlapped coding sequences',
 'Unfiltered merged reads',
 'Trimmed reverse reads',
 'Protein coding amino acid sequences',
 'Trimmed forward reads QC summary',
 'Geno Ontology summary statistics',
 'InterProScan summary statistics',
 'Kegg Ontology summary statistics',
 'ORF summary statistics',
 'Pfam summary statistcs',
 'Merged contigs CDS I5 summary',
 'Merged contigs HMM summary',
 'Merged contigs GO summary',
 'Merged contigs InterProScan slim',
 'Merged contigs InterProScan',
 'Merged contigs KO summary',
 'Merged contigs PFAM summary',
 'Eggnog emapper summary',
 'RNA prediction for SSU_rRNA_bacteria - Rfam accesssion number: RF00177',
 'RNA prediction for LSU_rRNA_e

In [39]:
[k for k in met_json['@graph'] if k['@type'] == 'File' and k['name'] == 'FASTA formatted contig sequences']

[{'@id': './final.contigs.fa.bz2',
  '@type': 'File',
  'name': 'FASTA formatted contig sequences',
  'description': 'These are the assembled contig sequences from the merged reads in FASTA format',
  'downloadUrl': 'https://s3.mesocentre.uca.fr/mgf-data-products/files/md5/b4/1de79ad2ea5e565064d74362b76563',
  'encodingFormat': 'application/x-bzip2',
  'contentSize': '92742645'}]

In [8]:
data = extract_data_by_name(met_json)
r = requests.get(data['downloadUrl'])
print(r.status_code)

# both look the same, but plots are generated by scripts, will not display here
# display(HTML(r.content.decode('utf-8')))
HTML(r.content.decode('utf-8'))

200


fastp version:,0.20.0 (https://github.com/OpenGene/fastp)
sequencing:,paired end (151 cycles + 151 cycles)
mean length before filtering:,"141bp, 141bp"
duplication rate:,31.835543%
Insert size peak:,151
total reads:,111.108678 M
total bases:,15.742938 G
Q20 bases:,15.562663 G (98.854887%)
Q30 bases:,15.154671 G (96.263297%)
GC content:,44.619341%
total reads:,32.275282 M


In [8]:
# questionable for Gcolab, but locally works
import webbrowser
webbrowser.open_new_tab(data['downloadUrl'])